In [ ]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
        .appName("Process Large JSON") \
        .master("local[*]") \
        .getOrCreate()

df = spark.read.json("review-Alabama.json")


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/25 21:20:49 WARN Utils: Your hostname, Asyrafs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.18.78 instead (on interface en0)
25/08/25 21:20:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/25 21:21:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df.printSchema()

root
 |-- gmap_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |-- rating: long (nullable = true)
 |-- resp: struct (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- time: long (nullable = true)
 |-- text: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)



In [3]:
df.show()

+--------------------+--------------------+----+------+--------------------+--------------------+-------------+--------------------+
|             gmap_id|                name|pics|rating|                resp|                text|         time|             user_id|
+--------------------+--------------------+----+------+--------------------+--------------------+-------------+--------------------+
|0x8862134e67ff5c8...|         Hadley Coan|NULL|     5|                NULL|Love this little ...|1626307876654|10337899798931755...|
|0x8862134e67ff5c8...|       Macey Johnson|NULL|     5|                NULL|The absolute BEST...|1626309762833|11259712068689102...|
|0x8862134e67ff5c8...|      Kelby Copeland|NULL|     5|                NULL|The absolute BEST...|1626301530535|10027036692530347...|
|0x8862134e67ff5c8...|         Wynn Thomas|NULL|     5|                NULL|The best children...|1626300613475|10447383783581786...|
|0x8862134e67ff5c8...|           Seth Meek|NULL|     1|{Hi Seth, my n

In [4]:
df1 = spark.read.json("review-Rhode_Island.json")

In [5]:
df1.show()

+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+--------------------+
|             gmap_id|                name|                pics|rating|                resp|                text|         time|             user_id|
+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+--------------------+
|0x89e44fe1c2a2a2c...|  Majestically child|                NULL|     5|{Thanks for comin...|Always wanted to ...|1600000495003|10223813496350060...|
|0x89e44fe1c2a2a2c...|    Kathryn Bartizek|[{[https://lh5.go...|     5|{Very glad to hea...|Mofongo, spanish ...|1602191331373|10899007040771662...|
|0x89e44fe1c2a2a2c...|         Rubia Bella|                NULL|     5|{Hope to see you ...|Very good! Will e...|1601590520320|10790690695529072...|
|0x89e44fe1c2a2a2c...|     angel olivencia|                NULL|     4|                NULL|Not bad place.

In [6]:
df2 = spark.read.json("review-California.json")

In [7]:
df2.show()

+--------------------+-----------------+----+------+----+---------------------------------+-------------+--------------------+
|             gmap_id|             name|pics|rating|resp|                             text|         time|             user_id|
+--------------------+-----------------+----+------+----+---------------------------------+-------------+--------------------+
|0x80c2c98c0e3c16f...|    Michael Rizal|NULL|     5|NULL|             Great company. Am...|1599164133778|11316555113047622...|
|0x80c2c98c0e3c16f...|Faranak Rafizadeh|NULL|     5|NULL|             Nice people\nhelpful|1618261672851|10122637137063761...|
|0x80c2c98c0e3c16f...|     Javier Perez|NULL|     5|NULL|                             NULL|1524515066787|11116770366698148...|
|0x80c2c98c0e3c16f...|          Luis P.|NULL|     5|NULL|                             NULL|1499739139293|11623088205734402...|
|0x80c2c98c0e3c16f...|   His Mama Cakez|NULL|     3|NULL|                             NULL|1621410940638|104181

In [14]:
spark.stop()